# Downsampling and BIDS conversion

## Introduction

In this section we will demonstrate how to prepare the data for further analysis. As the Quspin data are sampled at 1500 Hz the first step is to downsample to 500 Hz. Subsequenctly we will convert the data to MEG BIDS <i>(Niso et al., 2018)</i>
. BIDS (Brain Imaging Data Structure) is a standardised format for organising and describing neuroimaging datasets. It ensures that the meta data are present and that the file structure is consistent over data sets. 

## Preparation

Import the required modules:

In [1]:
import os
import numpy as np
import pandas as pd
import mne
from mne_bids import (
    BIDSPath,
    make_dataset_description,
    print_dir_tree,
    read_raw_bids,
    write_meg_calibration,
    write_meg_crosstalk,
    write_raw_bids
)

## File overview

The chapter relies on the input files:
~~~
<ROOT>/Main_split-01_meg.fif
<ROOT>/Main_split-02_meg.fif
~~~ 
and generate the out files:
~~~
<ROOT>/Main_fs.fif
<ROOT>/Main_event.fif

<ROOT/Ox_Sub1_BIDS/participants.json
<ROOT/Ox_Sub1_BIDS/participants.tsv
<ROOT/Ox_Sub1_BIDS/sub-01/ses-01/sub-01_ses-01_scans.tsv
<ROOT/Ox_Sub1_BIDS/dataset_description.json
<ROOT/Ox_Sub1_BIDS/sub-01/ses-01/meg/sub-01_ses-01_coordsystem.json
<ROOT/Ox_Sub1_BIDS/ses-01/meg/sub-01_ses-01_task-SpAtt_run-1_channels.tsv
<ROOT/Ox_Sub1_BIDS/ses-01/meg/sub-01_ses-01_task-SpAtt_run-1_events.json
<ROOT/Ox_Sub1_BIDS/meg/sub-01_ses-01_task-SpAtt_run-1_events.tsv
<ROOT/Ox_Sub1_BIDS/meg/sub-01_ses-01_task-SpAtt_run-1_meg.fif
<ROOT/Ox_Sub1_BIDS/sub-01_ses-01_task-SpAtt_run-1_meg.json
~~~

## Downloading the Data

Visit [OpenNeuro](https://openneuro.org/) and download the dataset.

## Loading the Data from Local Space

The QuSpin OPM data are stored in FIF format which is a binary file structure with embedded lables. As a first step we will load the data. Make sure the data path is set to where the data are downloaded. QuSpin systems save data across multiple split files based on the total size of the data.Here, both split files will be loaded into the workspace.However, we only need to specify the name of split-01, and MNE will automatically detect and load split-02 along with it
 We will also define the name of the resampled data with 'rs' added to the filename <code>(<a> *_rs.fif </a>)</code>

In [3]:
data_path = 'C:/Users/rakshita/Documents/Cerca_raw_files'
file_name = '20250410_110557_meg.fif'
raw_fname = os.path.join(data_path, file_name)

raw = mne.io.read_raw_fif(raw_fname, preload=True)
raw_resampled_fname = raw_fname.replace('.fif', f'_rs.fif')

event_fname = raw_fname.replace('.fif', f'_event.fif')

bids_folder = os.path.join(data_path, "Cerca_Spatt_BIDS")

Opening raw data file C:/Users/rakshita/Documents/Cerca_raw_files\20250410_110557_meg.fif...
    Range : 0 ... 2449499 =      0.000 ...  1632.999 secs
Ready.
Opening raw data file C:\Users\rakshita\Documents\Cerca_raw_files\20250410_110557_meg-1.fif...
    Range : 2449500 ... 3079615 =   1633.000 ...  2053.077 secs
Ready.
Reading 0 ... 3079615  =      0.000 ...  2053.077 secs...


In [4]:
print(raw.info)

<Info | 13 non-empty values
 bads: []
 ch_names: Trigger 1, Trigger 2, Trigger 3, Trigger 4, Trigger 5, Trigger ...
 chs: 27 Stimulus, 192 Magnetometers
 custom_ref_applied: False
 dev_head_t: MEG device -> head transform
 device_info: 2 items (dict)
 dig: 15603 items (3 Cardinal, 15600 Extra)
 file_id: 4 items (dict)
 highpass: 0.0 Hz
 line_freq: 0.0
 lowpass: 750.0 Hz
 meas_date: unspecified
 meas_id: 4 items (dict)
 nchan: 219
 projs: []
 sfreq: 1500.0 Hz
>


## Lowpass Filtering and Resampling

The data will now be resampled to 750 Hz. Prior to resampling data will be lowpass filtered at 750 Hz/3 = 250 Hz. The downsample is done to avoid alias problems *(Smith 1998, Ch. 3)*. After the lowpass filtering the data are resampled to the desired frequency.

In [5]:
desired_sfreq = 750
current_sfreq = raw.info['sfreq']

lowpass_freq = desired_sfreq / 3.0
raw_resampled = raw.copy().filter(l_freq=None, h_freq=lowpass_freq)

raw_resampled.resample(sfreq=desired_sfreq)

Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 2.5e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 250.00 Hz
- Upper transition bandwidth: 62.50 Hz (-6 dB cutoff frequency: 281.25 Hz)
- Filter length: 81 samples (0.054 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    6.9s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:   25.2s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:   55.2s


348 events found on stim channel Trigger 5
Event IDs: [1]
348 events found on stim channel Trigger 6
Event IDs: [1]
337 events found on stim channel Trigger 7
Event IDs: [1]
360 events found on stim channel Trigger 9
Event IDs: [1]
309 events found on stim channel Trigger 10
Event IDs: [1]


C:\Users\rakshita\AppData\Local\Temp\ipykernel_16708\2750611751.py:7: RuntimeWarning: Some events are duplicated in your different stim channels. 325 events were ignored during deduplication.
  raw_resampled.resample(sfreq=desired_sfreq)


348 events found on stim channel Trigger 5
Event IDs: [1]
348 events found on stim channel Trigger 6
Event IDs: [1]
337 events found on stim channel Trigger 7
Event IDs: [1]
360 events found on stim channel Trigger 9
Event IDs: [1]
309 events found on stim channel Trigger 10
Event IDs: [1]


C:\Users\rakshita\AppData\Local\Temp\ipykernel_16708\2750611751.py:7: RuntimeWarning: Some events are duplicated in your different stim channels. 331 events were ignored during deduplication.
  raw_resampled.resample(sfreq=desired_sfreq)
C:\Users\rakshita\AppData\Local\Temp\ipykernel_16708\2750611751.py:7: RuntimeWarning: Resampling of the stim channels caused event information to become unreliable. Consider finding events on the original data and passing the event matrix as a parameter.
  raw_resampled.resample(sfreq=desired_sfreq)


<Raw | 20250410_110557_meg.fif, 219 x 1539808 (2053.1 s), ~2.52 GiB, data loaded>

#### As a final step, the resampled data will be saved

In [6]:
raw_resampled.save(raw_resampled_fname, overwrite=True)

C:\Users\rakshita\AppData\Local\Temp\ipykernel_16708\649465886.py:1: RuntimeWarning: This filename (C:\Users\rakshita\Documents\Cerca_raw_files\20250410_110557_meg_rs.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_resampled.save(raw_resampled_fname, overwrite=True)


Writing C:\Users\rakshita\Documents\Cerca_raw_files\20250410_110557_meg_rs.fif
Closing C:\Users\rakshita\Documents\Cerca_raw_files\20250410_110557_meg_rs.fif
[done]


[WindowsPath('C:/Users/rakshita/Documents/Cerca_raw_files/20250410_110557_meg_rs.fif')]

#### To verify that the data has been resampled, print the following line and check the sampling frequency (sfreq) of the `.fif` file.


In [7]:
print(raw_resampled.info)

<Info | 13 non-empty values
 bads: []
 ch_names: Trigger 1, Trigger 2, Trigger 3, Trigger 4, Trigger 5, Trigger ...
 chs: 27 Stimulus, 192 Magnetometers
 custom_ref_applied: False
 dev_head_t: MEG device -> head transform
 device_info: 2 items (dict)
 dig: 15603 items (3 Cardinal, 15600 Extra)
 file_id: 4 items (dict)
 highpass: 0.0 Hz
 line_freq: 0.0
 lowpass: 250.0 Hz
 meas_date: unspecified
 meas_id: 4 items (dict)
 nchan: 219
 projs: []
 sfreq: 750.0 Hz
>


#### From now we will be working on the resampled data. Data with the 1500 Hz sample rate can be archived as they will no longer be used.

## Converting to MEG BIDS format

#### Read in the resampled data

In [8]:
del raw, raw_resampled
raw = mne.io.read_raw(raw_resampled_fname)

Opening raw data file C:/Users/rakshita/Documents/Cerca_raw_files\20250410_110557_meg_rs.fif...


C:\Users\rakshita\AppData\Local\Temp\ipykernel_16708\1265112872.py:2: RuntimeWarning: This filename (C:/Users/rakshita/Documents/Cerca_raw_files\20250410_110557_meg_rs.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw(raw_resampled_fname)


    Range : 0 ... 1539807 =      0.000 ...  2053.076 secs
Ready.


This code extracts the original trigger channels (Trigger 5 to Trigger 11) from the raw data. Each trigger channel represents a separate digital line. To reconstruct the full event information, the triggers are combined into a single digital trigger channel using a 4n4n weighting scheme, where nn is the channel index. Rising edges are detected to mark trigger onsets, and an MNE-compatible events array is created for subsequent analysis.

## Finding Triggers and write a FIF file

The BIDS data will include trigger information.The following code extracts the original trigger channels (Trigger 5 to Trigger 11) from the raw data. Each trigger channel represents a separate digital line. To reconstruct the full event information, the triggers are combined into a existing spare single digital trigger channel ('Trigger 1') using a $4^n$ weighting scheme, where $n$ is the channel index and an MNE-compatible events array is created for subsequent analysis. All the events extracted from the trigger channel are also included in an additional `.fif`file. 

In [9]:
raw.load_data()
trigger_chs =['Trigger 5','Trigger 6','Trigger 7','Trigger 8','Trigger 9','Trigger 10', 'Trigger 11']
trigger_data = raw.copy().pick_channels(trigger_chs).get_data()
n_times = trigger_data.shape[1]
combined_trigger = np.zeros(n_times, dtype=int)

for index_ch, ch_data in enumerate(trigger_data):
    combined_trigger += ch_data.astype(int) * (4 ** (index_ch + 1))
    
trigger1_idx = raw.ch_names.index('Trigger 1')

raw._data[trigger1_idx, :] = combined_trigger

events = mne.find_events(raw, stim_channel='Trigger 1', min_duration=0.003)
print(events[:10])  # Print first 10 events for verification

# Save events to a .fif file 
mne.write_events(event_fname, events, overwrite=True)

Reading 0 ... 1539807  =      0.000 ...  2053.076 secs...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
1365 events found on stim channel Trigger 1
Event IDs: [   4   16   64   68   80 1024 1028 1040 4096]
[[87906     0    68]
 [90367     0    16]
 [91270     0    64]
 [91674     0  1040]
 [91963     0  4096]
 [94080     0    16]
 [94982     0    64]
 [95412     0  1040]
 [95725     0  4096]
 [97880     0    16]]


C:\Users\rakshita\AppData\Local\Temp\ipykernel_16708\1797872278.py:18: RuntimeWarning: This filename (C:\Users\rakshita\Documents\Cerca_raw_files\20250410_110557_meg_event.fif) does not conform to MNE naming conventions. All events files should end with .eve, -eve.fif, -eve.fif.gz, -eve.lst, -eve.txt, _eve.fif, _eve.fif.gz, _eve.lst or _eve.txt
  mne.write_events(event_fname, events, overwrite=True)


## Naming the Events According to Trigger Information

The trigger values will now be assigned labels. This will be dependent on specific design of the study and care should be taken to assing informative labels.


In [10]:
event_dict = {
    'off': 0,
    'cue_Right': 4,       
    'cue_Left': 16,       
    'trial_Start': 20,
    'stimOnset': 64,      
    'catchOnset': 1024,     
    'dotOnRight': 1028,
    'dotOnLeft': 1040,
    'resp': 4096,           
    'blkStart': 68,      
    'blkEnd': 80,        
    'expEnd': 260,        
    'abort': 272,
    }

## Writing the BIDS File

For the BIDS conversion several parameters needs to be defined according to the subjection and session. Subsequently the BIDS structure can be written

In [11]:
raw.info["line_freq"] = 50
raw.set_annotations(None)
subject = '01'
session = '01'
task = 'SpAtt'
run = '01'

bids_path = BIDSPath(
    subject=subject, 
    session=session, 
    task=task, 
    run=run, 
    datatype="meg", 
    root=bids_folder
)
write_raw_bids(
    raw=raw,
    bids_path=bids_path,
    events=event_fname,
    event_id=event_dict,
    overwrite=True,
    allow_preload=True, 
    format='FIF',
    anonymize={'daysback': 50000, 'keep_his': False, 'keep_source': False}
)

Writing 'C:\Users\rakshita\Documents\Cerca_raw_files\Cerca_Spatt_BIDS\README'...
Writing 'C:\Users\rakshita\Documents\Cerca_raw_files\Cerca_Spatt_BIDS\participants.tsv'...
Writing 'C:\Users\rakshita\Documents\Cerca_raw_files\Cerca_Spatt_BIDS\participants.json'...
Writing 'C:\Users\rakshita\Documents\Cerca_raw_files\Cerca_Spatt_BIDS\sub-01\ses-01\meg\sub-01_ses-01_coordsystem.json'...
Writing 'C:\Users\rakshita\Documents\Cerca_raw_files\Cerca_Spatt_BIDS\sub-01\ses-01\meg\sub-01_ses-01_coordsystem.json'...


C:\Users\rakshita\AppData\Local\Temp\ipykernel_16708\2775216759.py:16: RuntimeWarning: This filename (C:/Users/rakshita/Documents/Cerca_raw_files\20250410_110557_meg_event.fif) does not conform to MNE naming conventions. All events files should end with .eve, -eve.fif, -eve.fif.gz, -eve.lst, -eve.txt, _eve.fif, _eve.fif.gz, _eve.lst, _eve.txt or -annot.fif
  write_raw_bids(


Used Annotations descriptions: ['blkEnd', 'blkStart', 'catchOnset', 'cue_Left', 'cue_Right', 'dotOnLeft', 'dotOnRight', 'resp', 'stimOnset']
Writing 'C:\Users\rakshita\Documents\Cerca_raw_files\Cerca_Spatt_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-SpAtt_run-01_events.tsv'...
Writing 'C:\Users\rakshita\Documents\Cerca_raw_files\Cerca_Spatt_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-SpAtt_run-01_events.json'...
Writing 'C:\Users\rakshita\Documents\Cerca_raw_files\Cerca_Spatt_BIDS\dataset_description.json'...
Writing 'C:\Users\rakshita\Documents\Cerca_raw_files\Cerca_Spatt_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-SpAtt_run-01_meg.json'...
Writing 'C:\Users\rakshita\Documents\Cerca_raw_files\Cerca_Spatt_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-SpAtt_run-01_channels.tsv'...
Copying data files to sub-01_ses-01_task-SpAtt_run-01_meg.fif
Reserving possible split file sub-01_ses-01_task-SpAtt_run-01_split-01_meg.fif
Writing C:\Users\rakshita\Documents\Cerca_raw_files\Cerca_Spatt_BIDS\sub-01\se

BIDSPath(
root: C:/Users/rakshita/Documents/Cerca_raw_files/Cerca_Spatt_BIDS
datatype: meg
basename: sub-01_ses-01_task-SpAtt_run-01_meg.fif)

#### As last check print the structure of the BIDS file

In [12]:
print_dir_tree(bids_folder)

|Cerca_Spatt_BIDS\
|--- README
|--- dataset_description.json
|--- participants.json
|--- participants.tsv
|--- sub-01\
|------ ses-01\
|--------- sub-01_ses-01_scans.tsv
|--------- meg\
|------------ sub-01_ses-01_coordsystem.json
|------------ sub-01_ses-01_task-SpAtt_run-01_channels.tsv
|------------ sub-01_ses-01_task-SpAtt_run-01_events.json
|------------ sub-01_ses-01_task-SpAtt_run-01_events.tsv
|------------ sub-01_ses-01_task-SpAtt_run-01_meg.fif
|------------ sub-01_ses-01_task-SpAtt_run-01_meg.json


## References

[1] Niso G, Gorgolewski KJ, Bock E, Brooks TL, Flandin G, Gramfort A, Henson RN, Jas M, Litvak V, Moreau JT, Oostenveld R, Schoffelen JM, Tadel F, Wexler J, Baillet S. MEG-BIDS, the brain imaging data structure extended to magnetoencephalography. *Scientific Data* 5, 180110 (2018). doi: [10.1038/sdata.2018.110](https://doi.org/10.1038/sdata.2018.110).

[2] Smith SW. *The Scientist and Engineer's Guide to Digital Signal Processing*. California Technical Publishing, 1998. [Link to PDF](https://www.dspguide.com/pdfbook.htm).
